In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from kret_studies import *
from kret_studies.notebook import *
from kret_studies.complex import *

logger = get_notebook_logger()

Loaded environment variables from /Users/Akseldkw/coding/Columbia/NNDL-Project/.env.


INFO:datasets:JAX version 0.7.2 available.


/Users/Akseldkw/coding/kretsinger/data/nb_log.log


In [3]:
from nndl_model.constants import DATA_DIR, MODEL_WEIGHT_DIR, ROOT_DIR

/Users/Akseldkw/micromamba/envs/kret_312/lib/python3.12/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/Users/Akseldkw/micromamba/envs/kret_312/lib/python3.12/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have 

In [4]:
DATA_DIR

PosixPath('/Users/Akseldkw/coding/Columbia/NNDL-Project/img_data')

In [5]:
out = uks_torch.exp_decay(1000)

In [6]:
DEVICE_TORCH_STR

'mps'

In [7]:
from nndl_model.impl.cnn_3_128_v000_test_aksel import CNN_3_128_V000_Test_Aksel

In [ ]:
# ---- sanity_sprint.py (or a single notebook cell) ----
import torch
import torch.nn as nn
import wandb

# Disable wandb logging for this quick check
wandb.init(project="nndl-sanity", mode="disabled")

from nndl_model.DATA_LOADERS import make_dataloaders
from nndl_model.BASE import BaseModel

# 1) Load data (uses DATA_DIR from your constants)
train_loader, val_loader, test_loader, M, S, K = make_dataloaders(
    batch_size=32,  # small, so it runs fast
    val_fraction=0.1,  # any small split
    image_size=224,
)

In [14]:
class TinyCNN(BaseModel):
    def __init__(self, lr=1e-3, feature_dim=64, S=3, K=10, M=None):
        super().__init__(lr=lr)  # 1) init base (no optimizer yet)
        self.model = nn.Sequential(  # 2) define backbone params
            nn.Conv2d(3, 32, 3, stride=2, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(32, 64, 3, stride=2, padding=1),
            nn.ReLU(inplace=True),
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten(),  # -> [B, 64]
        )
        # 3) add heads + optimizer now exists
        self.configure_hierarchy(feature_dim=feature_dim, num_super=S, num_sub=K, M=M)
        self.reset_optimizer()  # 4) create optimizer now that params exist
        # If you were NOT using hierarchy, call this instead:
        # self.reset_optimizer()

In [15]:
# 3) Instantiate model and sprint a couple epochs
model = TinyCNN(lr=1e-3, feature_dim=64, S=S, K=K, M=M)

KeyError: "attribute 'M' already exists"

In [ ]:
model.train_model(train_loader, val_loader, epochs=2)

# 4) Quick inference smoke test on the test loader (no labels there)
model.eval()
with torch.no_grad():
    for imgs, img_ids in test_loader:
        logits_sup, logits_sub = model(imgs.to(model.device))
        preds_super = logits_sup.argmax(1).cpu().tolist()
        preds_sub = logits_sub.argmax(1).cpu().tolist()
        print(f"Sample predictions: super={preds_super[:5]}, sub={preds_sub[:5]}")
        break  # just peek at one batch

ValueError: optimizer got an empty parameter list